# Notebook Instructions
<i>You can run the notebook document sequentially (one cell at a time) by pressing <b> shift + enter</b>. While a cell is running, a [*] will display on the left. When it has been run, a number will display indicating the order in which it was run in the notebook [8].</i>

<i>Enter edit mode by pressing <b>`Enter`</b> or using the mouse to click on a cell's editor area. Edit mode is indicated by a green cell border and a prompt showing in the editor area.</i> <BR>
    
This course is based on specific versions of python packages. You can find the details of the same in <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank" >this manual</a>. The manual also explains how to use these codes in other versions of python packages. <BR>

# Classifying Stocks- Screening Tool

In this notebook, you learn how to create a stock screener. 
1. Create a list of stocks
2. Find the regime on a relative basis for each stock
3. Identify the start date of the latest regime
4. Calculate absolute and relative cumulative returns


### Import libraries

In [1]:
# Data manipulation libraries
import numpy as np
import pandas as pd
from datetime import datetime

# Calculates swing highs and lows
from scipy.signal import *
from scipy.signal import argrelextrema

# Appends new file paths to import modules
import sys
sys.path.append("..")

# All functions library
from data_modules.short_selling import relative
from data_modules.short_selling import swings
from data_modules.short_selling import regime_fc
from data_modules.short_selling import cum_return_percent
from data_modules.short_selling import returns

# Imports charting libraries
import matplotlib.pyplot as plt

%matplotlib inline

### Import data

#### Files name:
1. BK_Jan_2010_to_Jan_2019, Bank of New York Mellon
2. JPM_Jan_2010_to_Jan_2019, JP Morgan
3. WFC_Jan_2010_to_Jan_2019, Wells Fargo
4. BAC_Jan_2010_to_Jan_2019, Bank of America
5. USDGBP_Jan_2010_to_Jan_2019.csv, contains forex data for 'USDGBP'
6. SP500_Jan_2010_to_Jan_2019.csv, contains benchmark data for 'SP500'

This data is available in the last unit of this course. You can find this file in the downloadables section of the course.

In [2]:
# Data is stored in the directory 'data_modules' 
relative_path = "../data_modules/"

In [3]:
# Import and read benchmark data using the function 'read_csv'
benchmark = pd.read_csv(relative_path +
                        'SP500_Jan_2010_to_Jan_2019.csv', index_col=0, parse_dates=True)

benchmark_name = 'SP500'

# Import and read forex data using the function 'read_data'
forex = pd.read_csv(relative_path +
                    'USDGBP_Jan_2010_to_Jan_2019.csv', index_col=0, parse_dates=True)

forex_name = 'USDGBP'

In [4]:
# Create a dataframe and dictionary list
market_regime = pd.DataFrame()
list_dict = []

### Stock classification for Bank of America, BAC.

In [5]:
# Read and process data
data = pd.read_csv(relative_path + 'BAC_Jan_2010_to_Jan_2019.csv',
                   index_col=0, parse_dates=True)

#### Find the stock's regime:

In [6]:
# Create a relative series
data = relative(stock_dataframe=data, benchmark_dataframe=benchmark, benchmark_name='SP500',
                forex_dataframe=forex, forex_name='USDGBP', decimals=2, start=None, end=None)

# Calculate the swings
data = swings(df=data, high='rebased_high',
              low='rebased_low', argrel_window=20)

# Calculate the stock regime
data = regime_fc(df=data, close='rebased_close', swing_low='srebased_low',
                 swing_high='srebased_high', threshold=1.5, t_dev=63, decimals=3)

# Current regime
regime = data['r_regime_floorceiling'][-1]

#### Find the regime change date:

In [7]:
# Find the latest regime change
regime_change_date = datetime.date(
    data[data['r_regime_change'].diff() != 0].index[-1])

#### Calculate absolute and relative cumulative returns:

In [8]:
# Calculate cumulative returns for relative series
cumulative_relative_returns = cum_return_percent(
    returns(data[regime_change_date:]['rebased_close']))[-1]

# Calculate cumulative returns for absolute series
cumulative_absolute_returns = cum_return_percent(
    returns(data[regime_change_date:]['Close']))[-1]

In [9]:
# Create a dictionary  of the columns required and append it to the list
row = {'file_name': data, 'regime': regime, 'regime_change_date': regime_change_date,
       'relative_returns': cumulative_relative_returns, 'absolute_returns': cumulative_absolute_returns}

# Append dictionary row to list
list_dict.append(row)

In [10]:
# Instantiate market_regime df using pd.DataFrame. from_dict()
market_regime = pd.DataFrame.from_dict(list_dict)
# Change the order of the columns
market_regime = market_regime[['file_name', 'regime_change_date',
                               'regime', 'relative_returns', 'absolute_returns']]
# Sort columns by regime change date
market_regime.sort_values(
    by=['regime_change_date'], ascending=False, inplace=True)

# Print the market regime
market_regime

,file_name,regime_change_date,regime,relative_returns,absolute_returns
0,Open High Low Close USDGBP...,2018-07-30,-1.0,-15.3,-20.3


Result: Regime for Bank of America is -1, bearish.

Now, let's perform these four steps over the other stock data by looping using a 'for' loop.

#### Create a stock universe:

Create a list of stock data, file_names[].
1. BK_Jan_2010_to_Jan_2019, Bank of New York Mellon
2. JPM_Jan_2010_to_Jan_2019, JP Morgan
3. WFC_Jan_2010_to_Jan_2019, Wells Fargo
4. BAC_Jan_2010_to_Jan_2019, Bank of America

In [11]:
# Create a list of stock data
file_names = ['BAC_Jan_2010_to_Jan_2019', 'BK_Jan_2010_to_Jan_2019',
              'JPM_Jan_2010_to_Jan_2019', 'WFC_Jan_2010_to_Jan_2019']

### Loop through the stocks using the same four steps:
1. Find the regime on a relative basis for each stock
2. Identify the start date of the latest regime
3. Calculate absolute and relative cumulative returns

In [12]:
list_dict = []
# For loop to iterate over the tickers list
for count, file_name in enumerate(file_names):

    # Data is stored in the directory 'data_modules'
    relative_path = "../data_modules/"
    # Read and process data: relative, swings, regime_fc
    df = pd.read_csv(relative_path + file_name + '.csv',
                     index_col=0, parse_dates=True)
    df = relative(stock_dataframe=df, benchmark_dataframe=benchmark, benchmark_name='SP500',
                  forex_dataframe=forex, forex_name='USDGBP', decimals=2, start=None, end=None)
    df = swings(df=df, high='rebased_high',
                low='rebased_low', argrel_window=20)
    df = regime_fc(df=df, close='rebased_close', swing_low='srebased_low',
                   swing_high='srebased_high', threshold=1.5, t_dev=63, decimals=3)

    # Instantiate variables with last row values for each field
    # Current regime
    regime = df['r_regime_floorceiling'][-1]

    regime_change_date = datetime.date(
        df[df['r_regime_change'].diff() != 0].index[-1])
    # Calculate cumulative returns for relative series
    cum_rel_rets = cum_return_percent(
        returns(df[regime_change_date:]['rebased_close']))[-1]
    # Calculate cumulative returns for relative series
    cum_abs_rets = cum_return_percent(
        returns(df[regime_change_date:]['Close']))[-1]

    # Create a row dictionary and list_dict.append(row)
    row = {'file_names': file_name, 'regime': regime, 'regime_change_date': regime_change_date,
           'relative_returns': cum_rel_rets, 'absolute_returns': cum_abs_rets}
    # Append dictionary row to list
    list_dict.append(row)

In [13]:
# Instantiate market_regime df using pd.DataFrame. from_dict()
market_regime = pd.DataFrame.from_dict(list_dict)
# Set Index as file_names
market_regime = market_regime.set_index('file_names')
# Change the order of the columns
market_regime = market_regime[['regime_change_date',
                               'regime', 'relative_returns', 'absolute_returns']]
# Sort columns by regime change date
market_regime.sort_values(
    by=['regime_change_date'], ascending=False, inplace=True)

# Print market regime
market_regime

,regime_change_date,regime,relative_returns,absolute_returns
file_names,,,,
BK_Jan_2010_to_Jan_2019,2018-10-10,-1.0,-1.5,-6.3
BAC_Jan_2010_to_Jan_2019,2018-07-30,-1.0,-15.3,-20.3
JPM_Jan_2010_to_Jan_2019,2018-07-30,-1.0,-9.6,-14.9
WFC_Jan_2010_to_Jan_2019,2018-01-24,1.0,-28.8,-28.3


### Conclusion
This notebook gives you a basic framework to create a powerful daily screening across your entire investment universe. You can scan for signals, calculate performance and perform all sorts of manipulations.
Bank of America and New York Mellon have a negative regime change, bearish. Whereas, Wells Fargo and JP Morgan have a positive regime change, bullish.
</span> <BR><BR>